## Surprise 라이브러리
+ 추천 시스템 개발을 위해 다양한 모델과 데이터 제공
+ scikit-learn과 사용방법이 유사

In [3]:
#!pip install scikit-surprise
import numpy as np
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [113]:
data = Dataset.load_builtin('ml-100k', prompt=False)

data.raw_ratings[:10]
#내장된 movielens 데이터로, 순서대로 사용자id, 상품id, 평점, 시간

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [116]:
import pandas as pd
df = pd.DataFrame(data.raw_ratings, columns=['user_id', 'movie_id', 'rating', 'timestamp'])

df

,user_id,movie_id,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596
...,...,...,...,...
99995,880,476,3.0,880175444
99996,716,204,5.0,879795543
99997,276,1090,1.0,874795795
99998,13,225,2.0,882399156


In [133]:
print('사용자 수 : {}'.format(df['user_id'].nunique()))
print('영화 수 : {}'.format(df['movie_id'].nunique()))
df.rating.value_counts()

사용자 수 : 943
영화 수 : 1682


4.0    34174
3.0    27145
5.0    21201
2.0    11370
1.0     6110
Name: rating, dtype: int64

## 추천 시스템
+ 크게 두가지로 구분 가능
    + 컨텐츠 기반 필터링(content-based filtering)
    + 협업 필터링(collaborative filtering)
    + 두가지를 조합한 hybrid 방식도 가능

## 1. 컨텐츠 기반 필터링
+ 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천  
    ex) 내가 점수를 높게 준 영화들과 유사한 영화를 추천
    => 즉, 컨텐츠 사이의 유사도를 기반으로 추천
+ 장점
    + 다른 사용자에 대한 정보가 없어도 됨
    + 대중적이지 않은 취향의 사용자에게도 추천 가능
    + 추천할 컨텐츠에 대한 설명 제공 가능
+ 단점
    + 유사도에 필요한 feature를 직접 설계해야 하기 때문에 도메인 지식 필요
    + 사용자의 기존 관심사항을 기반으로만 추천 가능  
        => 신규 사용자에게 추천하기가 힘듦
    + 너무 특정 관심사만 추천할 가능성(overspecialization)

## 2. 협업 필터링
+ 여러 대상을 이용해서 필터링하는 방식
+ 장점 
    + feature를 선택할 필요 없음
+ 단점
    + 학습에 충분한 사용자 수 필요(cold start)
    + 신규 아이템처럼 아무도 평점을 부여하지 않은 아이템은 추천할 수 없음(first rater)
    + 인기있는 아이템만 추천하는 경향이 있어(popularity bias) 소수 취향의 사용자에게 추천하기 힘듦
    + 사용자가 모든 아이템을 다 소비할 수도 없을 뿐더러, 소비한 아이템에 대한 평점 부여도 하지 않는 경우가 많기 때문에 사용자 또는 평점 행렬이 희소행렬인 경우가 많아 필터링이 어려울 수 있음


### 1. user-user collaborative filtering
+ 사용자를 고려해 추천하는 방법
+ 예를들어, 나와 비슷한 평점부여 방식을 가진 사용자를 찾아 그 사람이 선호한 영화 중 내가 아직 보지 않은 것을 추천

In [83]:
#어떤 원리인지 데이터로 확인해보기

data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)

raw_data[:,0] -= 1    #user_id (1~943) => 0~942
raw_data[:,1] -= 1    #movie_id (1~1682) => 0~1681

n_users = np.max(raw_data[:,0])
n_movies = np.max(raw_data[:,1])
shape = (n_users+1, n_movies+1)

shape

(943, 1682)

In [84]:
#우선, 명시적 피드백(rating)없이 시청목록만 가지고 유사한 사용자를 찾고, 추천리스트 만들어보기 
#user_id, movie_id 데이터를 인접행렬로 만들기

adj_matrix = np.ndarray(shape, dtype=int)   #(943, 1682) 모양의 n차원 행렬 생성(모든 값은 0)
for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] = 1.      #user_id, movie_id 데이터가 있는 위치의 값을 1로 변경
    
adj_matrix

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [85]:
# 나와 가장 유사한 사용자 찾는 방법 3가지
# 1. 이진 벡터의 내적을 통해 다른 사용자들과의 유사도 구하기

my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:   #내 id가 아닌 다른 id일 때
        similarity = np.dot(my_vector, user_vector)   
        if similarity > best_match:
            best_match = similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print(f'Best Match : {best_match}, Best Match Id : {best_match_id}')
print(f'Best Match Vector : {best_match_vector}')
#내 id가 0일 때, 나와 가장 유사한 사용자의 id는 275

#추천 리스트 출력
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:   #나는 안 본건데 나와 가장 유사한 사용자는 본 거라면
        recommend_list.append(i)
        
print(recommend_list)

Best Match : 183, Best Match Id : 275
Best Match Vector : [1 1 1 ... 0 0 0]
[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709

In [87]:
# 2. 유클리드 거리를 이용해 다른 사용자들과의 유사도 구하기

my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:   
        euclidean_distance = np.sqrt(np.sum(np.square(my_vector - user_vector)))
        if euclidean_distance < best_match:
            best_match = euclidean_distance
            best_match_id = user_id
            best_match_vector = user_vector
            
print(f'Best Match : {best_match}, Best Match Id : {best_match_id}')
print(f'Best Match Vector : {best_match_vector}')
#내 id가 0일 때, 나와 가장 유사한 사용자의 id는 737

recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:   
        recommend_list.append(i)
        
print(recommend_list)

Best Match : 14.832396974191326, Best Match Id : 737
Best Match Vector : [1 1 0 ... 0 0 0]
[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


In [88]:
# 3. 코사인 유사도를 사용해 추천(cos = (A*B) / (||A||*||B||))
def compute_cos_similarity(v1, v2):       #두 벡터 사이의 각도 이용
    norm1 = np.sqrt(np.sum(np.square(v1)))  
    norm2 = np.sqrt(np.sum(np.square(v2)))
    dot = np.dot(v1, v2)  
    return dot / (norm1 * norm2)

my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:   
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print(f'Best Match : {best_match}, Best Match Id : {best_match_id}')
print(f'Best Match Vector : {best_match_vector}')
#내 id가 0일 때, 나와 가장 유사한 사용자의 id는 915

recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:   
        recommend_list.append(i)
        
print(recommend_list)

Best Match : 0.5278586163659506, Best Match Id : 915
Best Match Vector : [1 1 1 ... 0 0 1]
[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


### 2. item-item collaborative filtering
+ 아이템을 고려해 추천하는 방법
+ 예를들어, 각 아이템의 벡터에 사용자들이 부여한 평점 정보가 담겨있을 때 아이템 A와 유사한 벡터를 가지는 아이템들의 평점 정보를 기반으로 아이템 A의 평점을 예측  

=> 두가지 방법을 비교하자면 사용자의 개인차가 크기 때문에, item-item 방식이 user-user보다 협업 필터링이 더 잘되는 경향

In [93]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)

raw_data[:,0] -= 1    #user_id (1~943) => 0~942
raw_data[:,1] -= 1    #movie_id (1~1682) => 0~1681

n_users = np.max(raw_data[:,0])
n_movies = np.max(raw_data[:,1])
shape = (n_users+1, n_movies+1)

adj_matrix = np.ndarray(shape, dtype=int)   #(943, 1682) 모양의 n차원 행렬 생성(모든 값은 0)
for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] = rating      #user_id, movie_id 데이터가 있는 위치의 값을 사용자가 평가한 영화점수로 변경
    
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [94]:
# 4-1. 이진 벡터의 내적을 통해 다른 사용자들과의 유사도 구하기

my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:   #내 id가 아닌 다른 id일 때
        similarity = np.dot(my_vector, user_vector)   
        if similarity > best_match:
            best_match = similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print(f'Best Match : {best_match}, Best Match Id : {best_match_id}')
print(f'Best Match Vector : {best_match_vector}')
#내 id가 0일 때, 나와 가장 유사한 사용자의 id는 아까와 같지만, Best Match와 Best Match Vector는 달라짐

recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0.: 
        recommend_list.append(i)
        
print(recommend_list)

Best Match : 2719, Best Match Id : 275
Best Match Vector : [5 4 3 ... 0 0 0]
[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 70

In [41]:
# 4-2. 유클리드 거리를 이용해 나와 유사한 사용자 찾기
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:   
        euclidean_distance = np.sqrt(np.sum(np.square(my_vector - user_vector)))
        if euclidean_distance < best_match:
            best_match = euclidean_distance
            best_match_id = user_id
            best_match_vector = user_vector
            
print(f'Best Match : {best_match}, Best Match Id : {best_match_id}')
print(f'Best Match Vector : {best_match_vector}')
#내 id가 0일 때, 나와 가장 유사한 사용자의 id는 아까와 같지만, Best Match와 Best Match Vector는 달라짐

recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:   
        recommend_list.append(i)
        
print(recommend_list)

Best Match : 55.06359959174482, Best Match Id : 737
Best Match Vector : [5 3 0 ... 0 0 0]
[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


In [42]:
# 4-3. 코사인 유사도를 사용해 추천(cos = (A*B) / (||A||*||B||))
def compute_cos_similarity(v1, v2):       #두 벡터 사이의 각도 이용
    norm1 = np.sqrt(np.sum(np.square(v1)))  
    norm2 = np.sqrt(np.sum(np.square(v2)))
    dot = np.dot(v1, v2)  
    return dot / (norm1 * norm2)

my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:   
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print(f'Best Match : {best_match}, Best Match Id : {best_match_id}')
print(f'Best Match Vector : {best_match_vector}')
#내 id가 0일 때, 나와 가장 유사한 사용자의 id는 아까와 같지만, Best Match와 Best Match Vector는 달라짐

recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:   
        recommend_list.append(i)
        
print(recommend_list)

Best Match : 0.569065731527988, Best Match Id : 915
Best Match Vector : [4 3 3 ... 0 0 3]
[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


## 잠재요인 협업필터링
+ 훈련데이터셋을 모델에 fit
+ 시험데이터셋을 모델에 test하게 되면 시험데이터셋 전체에 대해 추천을 수행
    + uid(유저id), iid(영화id), r_ui(실제 평점), est(예측 평점)
+ uid, iid를 문자열 형태로 모델에 넣어 predict하게 되면 개별 user와 개별 item에 대한 예측 평점을 return

In [4]:
from surprise import KNNBasic, SVD, SVDpp, NMF
from surprise.model_selection import train_test_split

data = Dataset.load_builtin('ml-100k', prompt=False)
train, test = train_test_split(data, test_size=0.2, random_state=2)

In [5]:
#KNN
model = KNNBasic()

model.fit(train)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [7]:
predictions = model.test(test)

print('prediction type : ', type(predictions), 'size : ', len(predictions))
predictions[:5]    #prediction 결과의 최초 5개 추출

prediction type :  <class 'list'> size :  20000


[Prediction(uid='522', iid='514', r_ui=2.0, est=4.41420305207026, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='56', iid='68', r_ui=3.0, est=3.692218120623278, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='880', iid='55', r_ui=3.0, est=3.805506251043342, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='370', iid='511', r_ui=4.0, est=4.566977658765549, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='109', iid='168', r_ui=3.0, est=3.880322542927191, details={'actual_k': 40, 'was_impossible': False})]

In [8]:
#100번 user가 300번 영화에 몇점을 줬을지 예측해보기
uid, iid = str(100), str(300)
model.predict(uid, iid) 
#실제 평점은 없으며, 예상평점은 3.56

Prediction(uid='100', iid='300', r_ui=None, est=3.562872379174314, details={'actual_k': 40, 'was_impossible': False})

In [9]:
#평가
scores = cross_validate(model, data, measures=['rmse', 'mae'],
                        cv=10, n_jobs=8, verbose=True)

Evaluating RMSE, MAE of algorithm KNNBasic on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.9698  0.9748  0.9757  0.9702  0.9731  0.9738  0.9841  0.9592  0.9756  0.9678  0.9724  0.0061  
MAE (testset)     0.7649  0.7689  0.7702  0.7669  0.7710  0.7682  0.7732  0.7578  0.7684  0.7645  0.7674  0.0041  
Fit time          0.46    0.56    0.59    0.70    0.66    0.63    0.64    0.64    0.60    0.51    0.60    0.07    
Test time         1.13    1.08    1.14    1.11    1.04    0.97    0.94    0.86    0.83    0.82    0.99    0.12    


In [45]:
#SVD
model = SVD()
scores = cross_validate(model, data, measures=['rmse', 'mae'],
                        cv=10, n_jobs=8, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.9198  0.9310  0.9207  0.9244  0.9294  0.9430  0.9318  0.9273  0.9361  0.9260  0.9289  0.0067  
MAE (testset)     0.7251  0.7380  0.7248  0.7248  0.7325  0.7453  0.7356  0.7315  0.7347  0.7316  0.7324  0.0062  
Fit time          3.59    3.71    3.75    3.61    3.60    3.60    3.58    3.48    3.57    3.56    3.61    0.07    
Test time         0.08    0.04    0.04    0.04    0.04    0.08    0.04    0.09    0.04    0.04    0.05    0.02    


In [47]:
#SVD++(시간 약 7-8분 걸림 주의,,)
model = SVDpp()
scores = cross_validate(model, data, measures=['rmse', 'mae'],
                        cv=10, n_jobs=8, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.9137  0.9154  0.9160  0.9220  0.9100  0.9181  0.9171  0.9134  0.9190  0.9030  0.9148  0.0050  
MAE (testset)     0.7176  0.7182  0.7172  0.7209  0.7129  0.7208  0.7188  0.7174  0.7190  0.7057  0.7169  0.0043  
Fit time          301.61  303.79  300.01  306.33  301.30  299.81  299.47  301.37  145.90  145.75  270.53  62.38   
Test time         1.42    1.01    1.54    0.98    1.38    1.51    1.49    1.33    0.97    0.96    1.26    0.24    


In [48]:
#NMF
model = NMF()
scores = cross_validate(model, data, measures=['rmse', 'mae'],
                        cv=10, n_jobs=8, verbose=True) 

Evaluating RMSE, MAE of algorithm NMF on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.9573  0.9484  0.9738  0.9532  0.9612  0.9626  0.9548  0.9516  0.9525  0.9599  0.9575  0.0069  
MAE (testset)     0.7536  0.7452  0.7686  0.7507  0.7581  0.7588  0.7515  0.7484  0.7506  0.7529  0.7538  0.0063  
Fit time          4.44    4.63    4.61    4.91    4.63    4.76    4.81    4.82    3.27    3.19    4.41    0.60    
Test time         0.06    0.08    0.08    0.07    0.08    0.06    0.04    0.03    0.04    0.03    0.06    0.02    


## Hybrid
+ 컨텐츠 기반 필터링 + 협업 필터링
+ 종류가 매우 다양함

In [49]:
from sklearn.decomposition import randomized_svd, non_negative_factorization

data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)

raw_data[:,0] -= 1    #user_id (1~943) => 0~942
raw_data[:,1] -= 1    #movie_id (1~1682) => 0~1681

n_users = np.max(raw_data[:,0])
n_movies = np.max(raw_data[:,1])
shape = (n_users+1, n_movies+1)

shape

(943, 1682)

### 1. randomized_svd를 사용한 하이브리드 추천
#### 1-1. 사용자 기반 추천 
+ 나와 비슷한 취향을 가진 다른 사용자의 목록을 추천

In [50]:
adj_matrix = np.ndarray(shape, dtype=int)   #(943, 1682) 모양의 n차원 행렬 생성(모든 값은 0)
for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] = rating      #user_id, movie_id 데이터가 있는 위치의 값을 점수로 변경
    
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [56]:
#인접행렬을 랜덤 특이값분해
U, S, V = randomized_svd(adj_matrix, n_components=2, random_state=2)  
S = np.diag(S)   #(2,0) => (2,2) 대각행렬로 만들기

U.shape, S.shape, V.shape
#세 행렬을 다 곱하면 다시 (943, 1682)행렬로 복원 됨
#np.matmul(np.matmul(U,S), V).shape

((943, 2), (2, 2), (2, 1682))

In [134]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(U):
    if my_id != user_id:   
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print(f'Best Match : {best_match}, Best Match Id : {best_match_id}')
print(f'Best Match Vector : {best_match_vector}')
#Best Match 점수가 매우 좋아짐!

user_recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:   
        user_recommend_list.append(i)
        
print(user_recommend_list)

Best Match : 0.9999942291877827, Best Match Id : 235
Best Match Vector : [0.03467744 0.00326755]
[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


#### 1-2. 항목 기반 추천
+ 내가 본 항목과 비슷한 항목을 추천

In [135]:
my_id, my_vector = 0, V.T[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(V.T):
    if my_id != user_id:   
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print(f'Best Match : {best_match}, Best Match Id : {best_match_id}')
print(f'Best Match Vector : {best_match_vector}')
#Best Match 점수가 매우 좋아짐!

item_recommend_list = []
for i, user_vector in enumerate(zip(adj_matrix)):
    if adj_matrix[i][my_id] > 0.9:   
        item_recommend_list.append(i)
        
print(item_recommend_list)

Best Match : 0.9999999948616307, Best Match Id : 1287
Best Match Vector : [ 0.00193695 -0.00176146]
[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 

In [136]:
#사용자 기반 추천결과와 항목 기반 추천결과에 중복되는 내용만 추천한다면?
recommend_list = []
for i in user_recommend_list:
    if i in item_recommend_list:
        recommend_list.append(i)
        
print(len(user_recommend_list),'+', len(item_recommend_list),'=>', len(recommend_list))
recommend_list

69 + 452 => 32


[273,
 274,
 285,
 288,
 293,
 297,
 306,
 312,
 410,
 418,
 428,
 431,
 434,
 477,
 482,
 504,
 520,
 522,
 525,
 531,
 548,
 613,
 631,
 654,
 660,
 691,
 698,
 704,
 734,
 755,
 863,
 933]

### 2. non_negative_factorization을 사용한 하이브리드 추천
#### 2-1. 사용자 기반 추천

In [137]:
A, B, iter = non_negative_factorization(adj_matrix, n_components=2, random_state=2)
A.shape, B.shape, iter

C:\Users\yeonok\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


((943, 2), (2, 1682), 78)

In [139]:
#사용자 기반 추천
my_id, my_vector = 0, A[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(A):
    if my_id != user_id:   
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print(f'Best Match : {best_match}, Best Match Id : {best_match_id}')
print(f'Best Match Vector : {best_match_vector}')
#Best Match 점수가 매우 좋아짐!

user_recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:   
        user_recommend_list.append(i)
        
print(user_recommend_list)

Best Match : 0.999999685629941, Best Match Id : 314
Best Match Vector : [0.65091385 0.31887431]
[272, 275, 284, 285, 287, 300, 301, 302, 304, 317, 323, 326, 339, 381, 427, 430, 432, 460, 465, 474, 503, 507, 512, 519, 522, 530, 602, 641, 644, 650, 653, 656, 672, 708, 731, 740, 745, 769, 791, 1064, 1083]


#### 2-2. 항목 기반 추천

In [140]:
my_id, my_vector = 0, B.T[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(B.T):
    if my_id != user_id:   
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print(f'Best Match : {best_match}, Best Match Id : {best_match_id}')
print(f'Best Match Vector : {best_match_vector}')
#Best Match 점수가 매우 좋아짐!

item_recommend_list = []
for i, user_vector in enumerate(zip(adj_matrix)):
    if adj_matrix[i][my_id] > 0.9:   
        item_recommend_list.append(i)
        
print(item_recommend_list)

Best Match : 0.9999999946284721, Best Match Id : 255
Best Match Vector : [0.04955243 0.07826838]
[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400

In [141]:
#사용자 기반 추천결과와 항목 기반 추천결과에 중복되는 내용만 추천한다면?
recommend_list = []
for i in user_recommend_list:
    if i in item_recommend_list:
        recommend_list.append(i)
        
print(len(user_recommend_list),'+', len(item_recommend_list),'=>', len(recommend_list))
recommend_list

41 + 452 => 17


[275,
 285,
 300,
 302,
 304,
 323,
 326,
 339,
 507,
 522,
 641,
 653,
 656,
 708,
 745,
 769,
 791]